In [ ]:
import os
import subprocess
import yaml

dot_path = r'C:\Program Files (x86)\Graphviz2.38\bin\dot.exe'
gv_path = r'C:\Users\entrup\Documents\Jupyter-Output\dependency_graph.gv'

In [ ]:
def parse_dependencies(path_):
    with open(path_) as stream:
        data_loaded = yaml.load(stream)
        name = data_loaded.get('name', None)
        if not name:
            return None
        deps = data_loaded.get('dependencies', None)
        if deps:
            deps_plugins = deps.get('plugins', None)
            if deps_plugins and deps_plugins != [None]:
                return deps_plugins
        

def analyse_dependencies(directory=r'C:\Users\entrup\workspace\PLX_methods_worksheets\trunk\__productstatus__'):
    dependencies = {}
    
    for root, dirs, files in os.walk(directory):
        for _file in files:
            if _file.endswith('.yml'):
                full_path = os.path.join(root, _file)
                result = parse_dependencies(full_path)
                plugin = os.path.basename(root)
                customer = os.path.basename((os.path.dirname(root)))
                dependencies.update({
                    "{}\\{}".format(customer, plugin): result
                })
    return dependencies

dep_result = analyse_dependencies()

In [ ]:
def remove_unconnected(dep_dict):
    result_dict = dep_dict.copy()
    connected = set()
    unconnected = []
    for dep_list in dep_dict.values():
        if dep_list:
            connected.update(dep_list)
    connected = [entry.lower() for entry in connected]
    for key in dep_dict.keys():
        if key in list(connected):
            print key
        if not dep_dict[key] and key.lower() not in connected:
            result_dict.pop(key)
            unconnected.append(key)
    return result_dict, unconnected

def optimize_name(name):
    name_opt = name.lower().replace(" ", "_").replace("-", "_").replace("\\", "_").replace(".", "_")
    return name_opt

def make_dependency_graph(dep_dict):
    digraph_str = "digraph Dependencies {\n"
    digraph_define = []
    digraph_connect = []
    for name in dep_dict.keys(): 
        name_opt = optimize_name(name)  
        digraph_define.append("\t{} [label=\"{}\"];".format(name_opt, name))
        deps = dep_dict[name]
        if deps:
            deps_opt = "; ".join([optimize_name(name) for name in deps])
            digraph_connect.append("\t{} -> {{{}}};".format(name_opt, deps_opt))
    digraph_define.sort()
    digraph_connect.sort()
    digraph_str += "\n".join(digraph_define) + "\n\n" + "\n".join(digraph_connect)
    digraph_str += "\n}"
    with open(gv_path, 'w') as f:
        f.write(digraph_str.replace('\\', '\\\\'))

filtered_dep_result, unconnected = remove_unconnected(dep_result)
print("Plugins without connection:")
for entry in sorted(unconnected):
    print("- {}".format(entry))
make_dependency_graph(filtered_dep_result)

In [ ]:
parameters = [dot_path, '-T', 'pdf', gv_path, '-O']
subprocess.call(parameters)